# building a ReAct agent

in our last notebook (`00_llm_call.ipynb`), we learned how to make basic LLM calls with function calling. Now we'll implement the **ReAct (Reasoning and Acting) pattern** - a fundamental building block for AI agents also known as llm in feedback loop. to build really good agentic system you need figure how to manage context without that the agent's performance will degrade for long horizon tasks way before reaching the max context lenght.(even for model with 1M context length). 

- in our agent hakken we have use the peristent todo list for managing the context for long coding tasks.
- and also we have long-term memory feature which save memory across session do it does not need to start fresh


### what is [ReAct](https://arxiv.org/pdf/2210.03629)?

ReAct stands for **Reasoning** and **Acting**. It's a pattern where the agent:
1. **Reasons** about what it needs to do
2. **Acts** by calling tools/functions  
3. **Observes** the results
4. **Repeats** until the task is complete

this creates a cycle: Think → Act → Observe → Think → Act → Observe...


by the end of this notebook, you'll understand:
- how to implement a ReAct loop
- how to manage conversation state across multiple iterations  
- how to handle tool execution and responses
- the foundation for building more sophisticated agents


In [17]:
# Import everything we need from the previous notebook
import json
import requests
from openai import OpenAI
from tavily import TavilyClient
import os
from dotenv import load_dotenv
from typing import Literal, List, Dict, Any

load_dotenv()

# Setup clients
openrouter_key = os.getenv("OPENROUTER_API_KEY")
openai_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_key,
)

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
model_name = os.getenv("OPENAI_MODEL")


## tools 

In [18]:
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "technology"] = "technology",
    include_raw_content: bool = False,
):
    search_docs = tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_docs

def calculate_math(expression: str):
    try:
        allowed_chars = set('0123456789+-*/.() ')
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return {"result": result, "expression": expression}
        else:
            return {"error": "Invalid characters in expression"}
    except Exception as e:
        return {"error": str(e)}

def write_note(content: str, filename: str = "note.txt"):
    try:
        with open(filename, 'w') as f:
            f.write(content)
        return {"success": True, "message": f"Note written to {filename}"}
    except Exception as e:
        return {"error": str(e)}

def read_note(filename: str = "note.txt"):
    try:
        with open(filename, 'r') as f:
            content = f.read()
        return {"content": content, "filename": filename}
    except Exception as e:
        return {"error": str(e)}


## tool description. 

In [19]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "internet_search",
            "description": "Search the internet for current information, news, and research.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query"},
                    "max_results": {"type": "integer", "description": "Max results (default: 5)", "default": 5},
                    "topic": {"type": "string", "enum": ["general", "news", "technology"], "default": "technology"},
                    "include_raw_content": {"type": "boolean", "default": False}
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function", 
        "function": {
            "name": "calculate_math",
            "description": "Calculate mathematical expressions safely",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "Mathematical expression to calculate"}
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "write_note", 
            "description": "Write content to a file",
            "parameters": {
                "type": "object",
                "properties": {
                    "content": {"type": "string", "description": "Content to write"},
                    "filename": {"type": "string", "description": "Filename (default: note.txt)", "default": "note.txt"}
                },
                "required": ["content"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_note",
            "description": "Read content from a file", 
            "parameters": {
                "type": "object",
                "properties": {
                    "filename": {"type": "string", "description": "Filename to read (default: note.txt)", "default": "note.txt"}
                },
                "required": []
            }
        }
    }
]

TOOL_MAPPING = {
    "internet_search": internet_search,
    "calculate_math": calculate_math,
    "write_note": write_note,
    "read_note": read_note
}


## main agentic loop

In [20]:
class ReActAgent:
    def __init__(self, client, tools, tool_mapping, model_name):
        self.client = client
        self.tools = tools
        self.tool_mapping = tool_mapping
        self.model_name = model_name
        self.messages = []
        self.max_iterations = 10  
        
    def add_system_message(self, content: str):
        self.messages.append({
            "role": "system",
            "content": content
        })
    
    def add_user_message(self, content: str):
        self.messages.append({
            "role": "user", 
            "content": content
        })
    
    def _execute_tool_call(self, tool_call):
        tool_name = tool_call.function.name
        
        try:
            tool_args = json.loads(tool_call.function.arguments) ## this is the tool call arguments. 
            print(f"executing {tool_name} with args: {tool_args}")
            
            if tool_name in self.tool_mapping:
                result = self.tool_mapping[tool_name](**tool_args)
                print(f"✅ tool result: {result}")
                return result
            else:
                error_msg = f"unknown tool: {tool_name}"
                print(f"❌ {error_msg}")
                return {"error": error_msg}
                
        except Exception as e:
            error_msg = f"tool execution failed: {str(e)}"
            print(f"❌ {error_msg}")
            return {"error": error_msg}
    
    def _handle_tool_calls(self, response_message):
        if not hasattr(response_message, 'tool_calls') or not response_message.tool_calls:
            return False
            
        self.messages.append({
            "role": "assistant",
            "content": response_message.content,
            "tool_calls": response_message.tool_calls
        })
        
        for tool_call in response_message.tool_calls:
            result = self._execute_tool_call(tool_call)
            
            self.messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result, indent=2)
            })
        
        return True 
    
    def run(self, user_input: str) -> str:
        self.add_user_message(user_input)
        
        print(f"User: {user_input}")
        print("=" * 50)
        
        iteration = 0
        while iteration < self.max_iterations: # the main agentic loop.
            iteration += 1
            print(f"Iteration {iteration}")
            
            request = {
                "model": self.model_name,
                "messages": self.messages,
                "tools": self.tools
            }
            
            completion = self.client.chat.completions.create(**request) # no streaming response. 
            response_message = completion.choices[0].message
            
            print(f"🤖 Assistant: {response_message.content}")
            
            has_tool_calls = self._handle_tool_calls(response_message)
            
            if not has_tool_calls:
                print("done with work")
                return response_message.content
        
        return "Maximum iterations reached. Task may be incomplete."

## system prompt 

In [21]:
agent = ReActAgent(
    client=openai_client,
    tools=tools,
    tool_mapping=TOOL_MAPPING,
    model_name=model_name
)

system_prompt = """You are a helpful AI assistant that can use various tools to help users.

You have access to these tools:
- internet_search: Search the web for current information
- calculate_math: Perform mathematical calculations  
- write_note: Write content to files
- read_note: Read content from files

Use these tools strategically to complete user requests. Think step by step and explain your reasoning."""

agent.add_system_message(system_prompt)


## testing 

In [22]:
result = agent.run("What is 25 * 37 + 156?")
print(f"final result: {result}")


User: What is 25 * 37 + 156?
Iteration 1
🤖 Assistant: 
executing calculate_math with args: {'expression': '25 * 37 + 156'}
✅ tool result: {'result': 1081, 'expression': '25 * 37 + 156'}
Iteration 2
🤖 Assistant: **1081**

To verify:  
25 × 37 = 925  
925 + 156 = 1081
done with work
final result: **1081**

To verify:  
25 × 37 = 925  
925 + 156 = 1081


In [23]:
agent2 = ReActAgent(
    client=openai_client,
    tools=tools, 
    tool_mapping=TOOL_MAPPING,
    model_name=model_name
)
agent2.add_system_message(system_prompt)

result = agent2.run("""
Search for the latest AI agent developments in 2025, calculate how many months are left in 2025 from now (assume we're in September), 
and write a summary note to a file called 'ai_research.txt'
""")

print(f"Final Result: {result}")


User: 
Search for the latest AI agent developments in 2025, calculate how many months are left in 2025 from now (assume we're in September), 
and write a summary note to a file called 'ai_research.txt'

Iteration 1
🤖 Assistant: 
executing internet_search with args: {'query': 'latest AI agent developments 2025', 'max_results': 5, 'topic': 'technology'}
❌ tool execution failed: Invalid topic. Must be 'general', 'news', or 'finance'
executing calculate_math with args: {'expression': '12 - 8'}
✅ tool result: {'result': 4, 'expression': '12 - 8'}
Iteration 2
🤖 Assistant: 
executing internet_search with args: {'query': 'latest AI agent developments 2025', 'max_results': 5, 'topic': 'news'}
✅ tool result: {'query': 'latest AI agent developments 2025', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://markets.ft.com/data/announce/detail?dockey=600-202509220406BIZWIRE_USPRX____20250922_BW194426-1', 'title': 'New Omdia Analysis Shows Agentic AI Outpacing Gro

In [24]:
agent3 = ReActAgent(
    client=openai_client,
    tools=tools,
    tool_mapping=TOOL_MAPPING, 
    model_name=model_name
)
agent3.add_system_message(system_prompt)

result = agent3.run("Read the contents of 'ai_research.txt' and summarize it in 2 sentences")
print(f"Final Result: {result}")


User: Read the contents of 'ai_research.txt' and summarize it in 2 sentences
Iteration 1
🤖 Assistant: 
executing read_note with args: {'filename': 'ai_research.txt'}
✅ tool result: {'content': 'Latest AI Agent Developments in 2025:\\n\\n- Market Growth: According to Omdia, the enterprise agentic AI software market is projected to surge from $1.5 billion in 2025 to $41.8 billion by 2030. By 2030, agentic AI will represent 31% of the total generative AI market. Key use cases include automated code development ($8.2B by 2030) and virtual assistants ($7.7B).\\n\\n- HR Tech Innovations: At HR Tech 2025, Visier launched its Manager Agent for frontline manager support, and Phenom introduced a fraud detection agent to combat fake candidates and AI-generated responses.\\n\\n- Events and Discussions: TechCrunch Disrupt 2025 features an AI stage with leaders from Character.AI, Hugging Face, Runway, and others, covering generative AI, developer tools, autonomous vehicles, and more. In pharma, BIO 